In [0]:
%sh
#ls -al 
cd /Volumes/workspace/aero/flights
unzip flights.zip
ls -al

In [0]:
file_location = f'/Volumes/workspace/aero/flights/flights.csv'
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)


In [0]:
# df.createOrReplaceTempView("flights") - Temporary View
df.write.option('overwrite', True).saveAsTable('aero.flights')


In [0]:
%sql
-- drop table flights;
-- show tables
show tables from aero



TODO:
1. List top 20 shortest flights by AirTime. Make it unique
2. Show these results along with the full names of these airports (both Origin and Dest). This will require joining Flights with Airports twice!
3. Add also: Origin.longitude, Origin.latitude as well as Dest.longitude and Dest.latitude and then create a query that will merge all these locations into one table. This will allow us to visualize these locations on a map.
4. Count number of all planes
5. Create a list of carriers (Full name of airlines) with their average arrival delay and number of realized flights. Visualize these delays

In [0]:
%sql
select distinct
  --FlightNum, 
  --TailNum, 
  Origin, 
  Dest, 
  --FlightDate, 
  AirTime 
from aero.flights 
where AirTime is not null 
order by AirTime asc 
limit 20


In [0]:
%sql
create or replace temporary view shortest_flights as select distinct
  --FlightNum, 
  --TailNum, 
  Origin, 
  oa.AIRPORT as OriginAirport,
  oa.LATITUDE as OriginLat,
  oa.LONGITUDE as OriginLong,
  Dest, 
  da.AIRPORT as DestAirport,
  da.LATITUDE as DestLat,
  da.LONGITUDE as DestLong,
  --FlightDate, 
  AirTime 
from aero.flights f
join aero.airports as oa on f.Origin = oa.IATA_CODE
join aero.airports as da on f.Dest = da.IATA_CODE
where AirTime is not null 
order by AirTime asc 
limit 20

In [0]:
%sql
select * from shortest_flights

In [0]:
%sql
select Origin as IATA_code, OriginLat as Latitude, OriginLong as Longitude from shortest_flights
UNION
select Dest as IATA_code, DestLat as Latitude, DestLong as Longitude from shortest_flights

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select count(distinct tailnum) from aero.flights

In [0]:
%sql
select * from aero.airlines

In [0]:
%sql
-- Create a list of carriers (Full name of airlines) with their average arrival delay and number of realized flights. Visualize these delays

select 
  Carrier, 
  Airline, 
  round(avg(ArrDelay), 2) as avg_delay, 
  count(*) as num_flights
from aero.flights f
join aero.airlines al on al.IATA_CODE= f.Carrier
group by Carrier, Airline
order by avg_delay asc

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select round(avg(ArrDelay), 2) as AVG_delay, round(median(ArrDelay), 2) as MED_delay from aero.flights

In [0]:
%sql
select * from aero.flights

In [0]:
df = _sqldf
quants = [round(0.025*x, 3) for x in range(1, 41)]
print(quants)

quant_arrdelay = df.approxQuantile(['ArrDelay'], quants, 0.0)[0]
print(quant_arrdelay)
#display(df)
#display(df.groupBy('Carrier').agg({'ArrDelay': 'avg'}))

quant_matrix = list(zip(quants, quant_arrdelay))
print(quant_matrix)

df_quant_arrdelay = spark.createDataFrame(quant_matrix, ['Quantile', 'ArrDelay'])
display(df_quant_arrdelay)

Databricks visualization. Run in Databricks to view.